# dropna() upsampling using FAMD

In [31]:
### read training data###
import pandas as pd
dataset=pd.read_csv('Simulated_Data_Train.csv').dropna()

In [36]:
## ### upsampling ####
### upsampling
## return upsampled dataset

def upsample(data):
    from sklearn.utils import resample
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

In [37]:
data_up=upsample(dataset)

In [38]:
def famdfitdata(dataset,dataset2,num_comp,itera,eng='auto',seed=123):
    """
    dataset1: for training
    dataset2: To be operated
    num_comp(int): number of components
    itera(int): iterations
    eng: 'auto', 'sklearn' or 'fbpca'
    """
    import prince
    famd = prince.FAMD(
        n_components=num_comp,
        n_iter=itera,
        copy=True,
        check_input=True,
        engine=eng,       ## Can be "auto", 'sklearn', 'fbpca'
        random_state=seed)
    famdfit = famd.fit(dataset.drop('Default_ind', axis=1))
    result=famdfit.row_coordinates(dataset2)
    return(result)

In [39]:
train=famdfitdata(data_up,data_up,10,10,)

In [44]:
### read test dataset
test=pd.read_csv('Simulated_Data_Test.csv')
test.dropna(inplace=True)

In [45]:
test1=famdfitdata(data_up,test,10,10,)

In [42]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train, data_up.iloc[:,20])
logisticRegr.score(train, data_up.iloc[:,20])


0.7095361515774421

In [46]:
pred_labels = logisticRegr.predict(test1)
from sklearn.metrics import accuracy_score
accuracy_score( pred_labels,test.iloc[:,20])

0.45338472657191037

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test.iloc[:,20],pred_labels)

array([[1580, 2230],
       [  39,  302]])

# mean upsampling using FAMD

In [54]:
import pandas as pd
dfn=pd.read_csv('Unconditional_mean_impute_Test.csv')
dfn_test=pd.read_csv('Unconditional_mean_impute_Test.csv')

In [55]:
train1=famdfitdata(dfn,dfn,10,10,)

In [56]:
test2=famdfitdata(dfn,dfn_test,10,10,)

In [58]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'
clf2 = LogisticRegression(solver = 'lbfgs')
clf2.fit(train1, dfn.iloc[:,20])
clf2.score(train1, dfn.iloc[:,20])


0.9334

In [59]:
pred_label = clf2.predict(test2)
from sklearn.metrics import accuracy_score
accuracy_score( pred_label,dfn_test.iloc[:,20])

0.9334

In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix(dfn_test.iloc[:,20],pred_label)

array([[4568,   31],
       [ 302,   99]])